We define classes for the problem.

In [1]:
from typing import List
import numpy as np

class Car:
    def __init__(self,car_id: int, time_slots_at_charging_unit: List[int], required_energy: int)-> None :
        self.car_id = car_id
        self.time_slots_at_charging_unit = time_slots_at_charging_unit
        self.required_energy = required_energy

    def __str__(self) -> str:
        return f"Car {self.car_id}: \n" \
               f" at charging station at time slots {self.time_slots_at_charging_unit} \n "\
               f" requires {self.required_energy} energy units"
    

class ChargingUnit:
    def __init__(self, unit_name: str, num_charging_levels: int, num_time_slots: int) -> None:
        self.unit_name = unit_name
        self.num_charging_levels = num_charging_levels
        self.num_time_slots = num_time_slots
        
        self.cars_to_charge = []

    def __str__(self) -> str:
        info_cars_registered = ""
        for car in self.cars_to_charge:
            info_cars_registered = info_cars_registered + " " + car.car_id
        return "Charging unit with\n" \
            " charging levels: " \
            f"{list(range(self.num_charging_levels))[1:-1]}\n" \
            " time slots: " \
            f"{list(range(self.num_time_slots))[1:-1]}\n" \
            " cars to charge:" \
            + info_cars_registered
    
    def register_car_for_charging(self, car: Car) -> None:
        if max(car.time_slots_at_charging_unit) > self.num_time_slots - 1:
            raise ValueError("From car required time slots not compatible "
            " with charging unit.")
        self.cars_to_charge.append(car)

    def reset_cars_for_charging(self)->None:
        self.cars_to_charge = []

    def generate_constraint_matrix(self) -> np.ndarray:
        "when a car is at a charging station the matrix element is 1 otherwise 0"
        constraint_matrix = np.zeros(self.cars_to_charge, len(self.cars_to_charge)*self.num_time_slots)
        for row_index in range(0, len(self.cars_to_charge)):
            offset = row_index*self.num_time_slots
            cols = np.array(
            self.cars_to_charge[row_index].time_slots_at_charging_unit) # here the cars_to_charge is a list with elements of cars, those have this attribute
            constraint_matrix[row_index, offset+cols] = 1
        return constraint_matrix
    
    def generate_constraint_rhs(self) -> np.ndarray:
        """Vector with required energy as entries"""
        number_cars_to_charge = len(self.cars_to_charge)
        constraint_rhs = np.zeros((number_cars_to_charge, 1))
        for row_index in range(0, number_cars_to_charge):
            constraint_rhs[row_index] = self.cars_to_charge[row_index].required_energy # here the cars_to_charge is a list with elements of cars, those have this attribute
        return constraint_rhs
    
    def generate_cost_matrix(self) -> np.ndarray:
        number_cars_to_charge = len(self.cars_to_charge)
        return np.kron(
        np.ones((number_cars_to_charge, 1)) \
        @ np.ones((1, number_cars_to_charge)),
        np.eye(self.number_time_slots))

Creating car instances as follows

In [2]:
car_green = Car(car_id="car_green", time_slots_at_charging_unit=[0,1,2,3], required_energy=8)
car_red = Car(car_id="car_red", time_slots_at_charging_unit=[0,1,2,3,4,5,6], required_energy=12)

In [7]:
car_green.required_energy

8

In [15]:
print(car_green,'\n',car_red)

Car car_green: 
 at charging station at time slots [0, 1, 2, 3] 
  requires 8 energy units 
 Car car_red: 
 at charging station at time slots [0, 1, 2, 3, 4, 5, 6] 
  requires 12 energy units


Testing the charging unit class